In [3]:
!pip install tensorflow=="2.13.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 15.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attem

In [4]:
# Import library yang diperlukan
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from PIL import Image

In [5]:
# Mendefinisikan parameter-parameter
batch_size = 32
epochs = 1  # Meningkatkan jumlah epochs
image_height = 150
image_width = 150
train_data_dir = '/kaggle/input/dataset-capstone-sortsavvy/archive/DATASET/TRAIN'
validation_data_dir = '/kaggle/input/dataset-capstone-sortsavvy/archive/DATASET/TEST'
test_image_path = '/kaggle/input/dataset-capstone-sortsavvy/archive/DATASET/TEST/O/O_12590.jpg'  # Path untuk gambar uji

In [6]:
from PIL import Image

# Contoh penggunaan Pillow
image = Image.open('/kaggle/input/dataset-capstone-sortsavvy/archive/DATASET/TEST/O/O_12590.jpg')
image.show()


In [7]:
# Membuat objek ImageDataGenerator untuk augmentasi data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   rotation_range=40,  # Menambahkan augmentasi rotasi
                                   width_shift_range=0.2,  # Menambahkan augmentasi pergeseran lebar
                                   height_shift_range=0.2,  # Menambahkan augmentasi pergeseran tinggi
                                   brightness_range=[0.8, 1.2],  # Augmentasi kecerahan
                                   fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
# Memuat data latih dan data validasi
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(image_height, image_width),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size=(image_height, image_width),
                                                              batch_size=batch_size,
                                                              class_mode='binary')

Error: no "view" mailcap rules found for type "image/png"
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening '/tmp/tmp4z6myt5l.PNG'


Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [9]:
# Membangun arsitektur CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3), kernel_regularizer=l2(0.001)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),  # Menambahkan layer Conv2D
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),  # Menambahkan dropout layer
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Menambahkan dense layer
    Dense(1, activation='sigmoid')
])

In [10]:
# Compile model dengan learning rate yang lebih rendah
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
# Callback untuk early stopping dan mengurangi learning rate ketika mengalami plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

In [12]:
pip install scipy

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [52]:
# # Melatih model
# history = model.fit(train_generator,
#                     steps_per_epoch=train_generator.samples // batch_size,
#                     epochs=epochs,
#                     validation_data=validation_generator,
#                     validation_steps=validation_generator.samples // batch_size,
#                     callbacks=[early_stopping, reduce_lr])

In [53]:
# # Simpan model sebagai mode.h5
# model.save('model.h5')

In [13]:
# simpan model h5 pakai method export()
model.export('/kaggle/working/model_sort_savvy_export.h5')

Saved artifact at '/kaggle/working/model_sort_savvy_export.h5'. The following endpoints are available:

* Endpoint 'serve'
  Args:
    args_0: float32 Tensor, shape=(None, 150, 150, 3)
  Returns:
    float32 Tensor, shape=(None, 1)


In [23]:
import tensorflow as tf

# Muat model SavedModel
saved_model_dir = '/kaggle/working/model_sort_savvy_export.h5'
model = tf.saved_model.load(saved_model_dir)

# Mendapatkan fungsi konkret dari model yang di-load
concrete_func = model.signatures['serving_default']

# Melihat detail dari fungsi konkret untuk mengetahui nama tensor output
print(concrete_func.structured_outputs)
print(concrete_func.structured_input_signature)

# Misalkan output tensor memiliki nama 'output_0'
output_tensor_name = list(concrete_func.structured_outputs.keys())[0]

@tf.function
def modified_concrete_func(inputs):
    # Memanggil fungsi asli
    original_output = concrete_func(**inputs)
    # Memodifikasi output shape, misalnya dengan reshape
    modified_output = tf.reshape(original_output[output_tensor_name], [1, 2])
    return {output_tensor_name: modified_output}

# Buat concrete function baru dari modified_concrete_func
input_signature = concrete_func.structured_input_signature[1]

# Definisikan fungsi konkret baru dengan signature yang tepat
modified_concrete_function = modified_concrete_func.get_concrete_function(input_signature)

# Konversi model ke TFLite
converter = tf.lite.TFLiteConverter.from_concrete_functions([modified_concrete_function])
tflite_model = converter.convert()

# Simpan model TFLite
with open('model_modified.tflite', 'wb') as f:
    f.write(tflite_model)


# xxxxxxxxxx
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

# Load the EfficientDet Lite0 model spec and modify its configuration
spec = model_spec.get('/kaggle/working/model_modified.tflite')
spec.config.max_output_size = 10

{'output_0': TensorSpec(shape=(None, 1), dtype=tf.float32, name='output_0')}
((), {'conv2d_input': TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='conv2d_input')})


ModuleNotFoundError: No module named 'tflite_model_maker'

In [56]:
# # Memuat model yang sudah dilatih
# model = load_model('model.h5')

In [15]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

In [58]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model('/kaggle/working/model_sort_savvy_export.h5')
tflite_model = converter.convert()

In [59]:
# Save the converted model to a file
tflite_model_path = '/kaggle/working/model_ML_sort_savvy.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [60]:
# # Load the existing trained model
# model_path = 'C:\DATA EGA\BANGKIT\Capstone\model.h5'
# try:
#     model = load_model(model_path)
#     print = ("Model Succesfully Loaded")
    
#     # Print Model Summary
#     model.summary()
# except Exception as e:
#     print(f"Error loading the model: {e}")
#     # Exit the program if loading the model fails
#     exit()

In [61]:
# # Buat converter
# converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [62]:
# from tensorflow.keras.models import load_model, save_model

# # Muat model Keras
# model = load_model('/kaggle/working/model.h5')

# # Simpan ulang model
# save_model(model, 'new_model.h5')

In [63]:
# import tensorflow as tf
# from tensorflow.keras.models import load_model

# # Muat model
# model = load_model('new_model.h5')

# # Buat converter
# converter = tf.lite.TFLiteConverter.from_keras_model(model)

# # Konversi model
# tflite_model = converter.convert()

# # Simpan model ke file .tflite
# with open('model.tflite', 'wb') as f:
#     f.write(tflite_model)


In [64]:
# import tensorflow as tf
# from tensorflow.keras.models import load_model

# # Muat model dari file .h5
# model = load_model('/kaggle/working/new_model.h5')

# # Simpan model sebagai SavedModel
# model.save('saved_model.keras')

# # Buat converter dari SavedModel
# converter = tf.lite.TFLiteConverter.from_keras_model(model)

# # Konversi model
# tflite_model = converter.convert()

# # Simpan model ke file .tflite
# with open('model.tflite', 'wb') as f:
#     f.write(tflite_model)

# # Output file .tflite berhasil disimpan
# print("Model TFLite berhasil disimpan.")


convert metadata
referensi: https://www.tensorflow.org/lite/models/convert/metadata_writer_tutorial#image_classifiers

In [16]:
pip install tflite_support_nightly

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 27.1 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install flatbuffers

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install tflite-support

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 26.2 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [68]:
# !pip install tensorflow=="2.13.0"
# udah di-install di atas sebelumnya

In [19]:
pip show tensorflow

Name: tensorflow
Version: 2.13.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: explainable-ai-sdk, tensorflow-cloud, tensorflow-decision-forests, tensorflow-serving-api, tensorflow-text, tf_keras, witwidget
Note: you may need to restart the kernel to use updated packages.


In [20]:
# Step 1: Import the necessary libraries
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata
import flatbuffers
import os

# Step 2: Define paths and create metadata writer
# _MODEL_PATH = "/kaggle/input/labelmap/model_ML_sort_savvy_new.tflite"
_LABEL_FILE = "/kaggle/input/dataset-capstone-sortsavvy/labelmap.txt"

_MODEL_PATH = "/kaggle/working/model_modified.tflite"
# _LABEL_FILE = "/kaggle/input/labelmap.txt"
_SAVE_TO_PATH = "/kaggle/working/model_1_2_sort_savvy_metadata.tflite"

ImageClassifierWriter = image_classifier.MetadataWriter

# Step 3: Populate metadata
writer = ImageClassifierWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5], [_LABEL_FILE]
)

# Save the model with metadata
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

# Step 4: Verify metadata
displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

Metadata populated:
{
  "name": "ImageClassifier",
  "description": "Identify the most prominent object in the image from a known set of categories.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be classified.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
        }
      ],
      "output_tensor_metadata": [
        {
    

inspect tflite model

In [21]:
import tensorflow as tf

# Load the TFLite model
model_path = "/kaggle/working/model_1_2_sort_savvy_metadata.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


RuntimeError: tensorflow/lite/kernels/reshape.cc:92 num_input_elements != num_output_elements (1 != 2)Node number 13 (RESHAPE) failed to prepare.Failed to apply the default TensorFlow Lite delegate indexed at 0.

In [22]:
import tensorflow as tf

model_path = "/kaggle/working/model_1_2_sort_savvy_metadata.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Check model input and output details
print("Input details:", input_details)
print("Output details:", output_details)

# Check model output
import numpy as np

input_shape = input_details[0]['shape']
dummy_input = np.random.random(input_shape).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], dummy_input)
interpreter.invoke()

# Get the output
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Output data:", output_data)

RuntimeError: tensorflow/lite/kernels/reshape.cc:92 num_input_elements != num_output_elements (1 != 2)Node number 13 (RESHAPE) failed to prepare.Failed to apply the default TensorFlow Lite delegate indexed at 0.